In [ ]:
import numpy as np
import random
import matplotlib.pyplot as plt
from pypokerengine.players import BasePokerPlayer
from pypokerengine.api.game import setup_config, start_poker
from pypokerengine.utils.card_utils import gen_cards, estimate_hole_card_win_rate
from pypokerengine.engine.hand_evaluator import HandEvaluator
from pypokerengine.engine.card import Card

In [ ]:
class HonestPlayer(BasePokerPlayer):
    def __init__(self, raise_wr = 0.6, call_wr = 0.3, nb_sim=100):
        super().__init__() 
        self.raise_wr = raise_wr
        self.call_wr = call_wr
        self.nb_sim = nb_sim
        
    def declare_action(self, valid_actions, hole_card, round_state):
        community_card = round_state['community_card']
        win_rate = estimate_hole_card_win_rate(
                nb_simulation=self.nb_sim,
                nb_player=self.nb_player,
                hole_card=gen_cards(hole_card),
                community_card=gen_cards(community_card)
                )
        if win_rate >= self.raise_wr:
            action = valid_actions[2]  # fetch bet action info
            valid_amounts = action['amount']
            raise_amount =  (valid_amounts['max'] - valid_amounts['min'])/ 5.0 + valid_amounts['min']
            return valid_actions[2]['action'], raise_amount
        elif win_rate >= self.call_wr:
            action = valid_actions[1]  # fetch CALL action info
        else:
            call_amount = valid_actions[1]['amount']
            if(call_amount == 0):
                return valid_actions[1]['action'], valid_actions[1]['amount']
            action = valid_actions[0]  # fetch FOLD action info
        return action['action'], action['amount']

    def receive_game_start_message(self, game_info):
        self.nb_player = game_info['player_num']

    def receive_round_start_message(self, round_count, hole_card, seats):
        pass

    def receive_street_start_message(self, street, round_state):
        pass

    def receive_game_update_message(self, action, round_state):
        pass

    def receive_round_result_message(self, winners, hand_info, round_state):
        pass

In [ ]:
class YourPokerBot(BasePokerPlayer):
    def __init__(self, raise_wr = 0.6, call_wr = 0.3, nb_sim=100):
        super().__init__() 
        
    def declare_action(self, valid_actions, hole_card, round_state):
        action = random.choice(valid_actions)
        if(action['action'] == 'raise'):
            valid_amounts = action['amount']
            raise_amount =  np.random.rand(1)[0]*(valid_amounts['max'] - valid_amounts['min']) + valid_amounts['min']
            amount = raise_amount
        else:
            amount = action['amount']
        return action['action'], amount

    def receive_game_start_message(self, game_info):
        self.nb_player = game_info['player_num']

    def receive_round_start_message(self, round_count, hole_card, seats):
        pass

    def receive_street_start_message(self, street, round_state):
        pass

    def receive_game_update_message(self, action, round_state):
        pass

    def receive_round_result_message(self, winners, hand_info, round_state):
        pass

In [ ]:
# Keeps track of the players' performances.
results_dict = {}
# Starting money for each game.
initial_stack = 1000
# Number of total games of poker to play.
num_games = 10
# Number of hands of poker to play per game.
hands_per_game = 50

# Run the games.
for i in range(num_games):
    # Setup the game rules.
    config = setup_config(max_round=hands_per_game, initial_stack=initial_stack, small_blind_amount=5)
    # Register the players.
    config.register_player(name="YourBot", algorithm=YourPokerBot())
    config.register_player(name="HonestBot1", algorithm=HonestPlayer(raise_wr = 0.9, call_wr=0.7, nb_sim=50))
    config.register_player(name="HonestBot2", algorithm=HonestPlayer(raise_wr = 0.7, call_wr=0.4, nb_sim=200))
    config.register_player(name="HonestBot3", algorithm=HonestPlayer(raise_wr = 0.3, call_wr=0.1, nb_sim=200))

    # Play the game.
    game_result = start_poker(config, verbose=1)

    # Track the results.
    players = game_result['players']
    for player in players:
        if(player['name'] not in results_dict.keys()):
            results_dict[player['name']] = []
        results_dict[player['name']] += [player['stack'] - initial_stack]


print(results_dict)

In [ ]:
def plot_running_sums(data):
  """Plots the running sums of a dictionary of arrays.

  Args:
    data: A dictionary where the keys are labels and the values are arrays of numbers.
  """

  for label, values in data.items():
    running_sum = np.cumsum(values)
    plt.plot(running_sum, label=label)

  plt.xlabel("Game #")
  plt.ylabel("Winnings ($)")
  plt.legend()
  plt.show()

plot_running_sums(results_dict)